In [67]:
import pandas as pd
import re
import ast

In [68]:
phi_df = pd.read_csv("wikipedia_with_phi_3.csv")

In [69]:
phi_df.columns

Index(['id', 'title', 'context', 'question', 'answers', 'phi_3'], dtype='object')

In [70]:
def remove_multiple_lines(val):
    if "\n" in val:
        val = val.split("\n")[0]
    return val.strip()

def format_answer(val):
    return ast.literal_eval((re.search(r"'text': array\((\[.*?\])", val).group(1)))[0]

phi_df['phi_3'] = phi_df['phi_3'].apply(remove_multiple_lines)
phi_df['answers'] = phi_df['answers'].apply(format_answer)

In [71]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/vfayos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [77]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

phi_df['bleu_score'] = 0
#phi_df['rouge_score'] = 0
phi_df['meteor_score'] = 0

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for index, row in phi_df.iterrows():
    bleu_scr = round(sentence_bleu([row['answers'].split(' ')], row['phi_3'].split(' ')), 4)
    #rougue_scr = round(rougue_score([row['answers'].split(' ')], row['phi_3'].split(' ')), 4)
    meteor_scr = round(meteor_score([row['answers'].split(' ')], row['phi_3'].split(' ')), 4)
    
    phi_df.at[index, 'bleu_score'] = bleu_scr
    #row['rouge_score'] = rougue_scr
    phi_df.at[index, 'meteor_score'] = meteor_scr


/tmp/ipykernel_32245/1330334429.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1724' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phi_df.at[index, 'meteor_score'] = meteor_scr
/tmp/ipykernel_32245/1330334429.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.218' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phi_df.at[index, 'bleu_score'] = bleu_scr
/home/vfayos/anaconda3/envs/bleu/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vfayos/anaconda3/envs

In [76]:
print(sum(phi_df['meteor_score'])/1000)

0.3439084
